

# Atividade de Tradução Português-Inglês com Transformer e Controle de Versões

Este Colab tem como objetivo praticar a criação e o treinamento de um modelo Transformer de **sequência para sequência** para traduzir do **português para o inglês**, com a adição de **controle de versões** para acompanhamento do progresso do projeto.

O **Transformer**, originalmente proposto no artigo ["Attention is All You Need"](https://arxiv.org/abs/1706.03762) por Vaswani et al. (2017), utiliza o mecanismo de **autoatenção**, permitindo que informações fluam eficientemente entre as sequências de entrada. Ele substitui arquiteturas mais tradicionais como **CNNs** e **RNNs**, mostrando desempenho superior em tarefas de tradução automática.

## Objetivos:

- Preparar um dataset de tradução (português-inglês) para treinar o modelo.
- Implementar componentes essenciais do Transformer:
  - Embeddings posicionais.
  - Camadas de atenção.
  - Codificador e decodificador.
- Treinar o Transformer.
- Avaliar a qualidade das traduções geradas.
- Realizar **controle de versões** para rastrear o progresso, testes e melhorias do modelo.

## Ferramentas e Tecnologias:

- **TensorFlow/Keras**: para a implementação e treinamento do modelo Transformer.
- **Git**: para o controle de versões, onde serão registrados commits que documentam o progresso e as alterações feitas.
- **Colab**: para execução do código em ambiente interativo e compartilhável.


<img src="https://www.tensorflow.org/images/tutorials/transformer/apply_the_transformer_to_machine_translation.gif" alt="Aplicando o Transformer à tradução automática">

**Figura 1**: Aplicando o Transformer à tradução automática. Fonte: [Google AI Blog](https://ai.googleblog.com/2017/08/transformer-novel-neural-network.html).

## Setup

Instalando o [TensorFlow Datasets](https://tensorflow.org/datasets) para carregar o conjunto de dados e o [TensorFlow Text](https://www.tensorflow.org/text) para o pré-processamento de texto:



In [ ]:
# Install the most re version of TensorFlow to use the improved
# masking support for `tf.keras.layers.MultiHeadAttention`.
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install protobuf~=3.20.3
!pip install -q tensorflow_datasets
!pip install -q -U tensorflow-text tensorflow

Importe os módulos necessários:

In [ ]:
import logging
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf

import tensorflow_text

## Manipulação de Dados

Esta seção faz o download do conjunto de dados e do tokenizador de subpalavras, a partir [deste tutorial](https://www.tensorflow.org/text/guide/subwords_tokenizer), e depois organiza tudo em um `tf.data.Dataset` para o treinamento.

<section class="expandable tfo-display-only-on-site">
<button type="button" class="button-red button expand-control">Alternar seção</button>

### Download do dataset

Usamos o TensorFlow Datasets para carregar o [conjunto de dados de tradução Português-Inglês](https://www.tensorflow.org/datasets/catalog/ted_hrlr_translate#ted_hrlr_translatept_to_en) do TED Talks Open Translation Project. Esse conjunto de dados contém aproximadamente 52.000 exemplos de treino, 1.200 para validação e 1.800 exemplos de teste.

In [ ]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en',
                               with_info=True,
                               as_supervised=True)

train_examples, val_examples = examples['train'], examples['validation']

O objeto `tf.data.Dataset` retornado pelo TensorFlow Datasets gera pares de exemplos de texto:

In [ ]:
for pt_examples, en_examples in train_examples.batch(3).take(1):
  print('> Exemplos em Português:')
  for pt in pt_examples.numpy():
    print(pt.decode('utf-8'))
  print()

  print('> Exemplos em Inglês:')
  for en in en_examples.numpy():
    print(en.decode('utf-8'))

### Configurar o tokenizador

Agora que carregamos o conjunto de dados, precisamos tokenizar o texto, de forma que cada elemento seja representado como um [token](https://developers.google.com/machine-learning/glossary#token) ou ID de token (uma representação numérica).

A tokenização é o processo de dividir o texto em "tokens". Dependendo do tokenizador, esses tokens podem representar pedaços de sentenças, palavras, subpalavras ou caracteres. Para saber mais sobre tokenização, visitamos [este guia](https://www.tensorflow.org/text/guide/tokenizers).

In [ ]:
model_name = 'ted_hrlr_translate_pt_en_converter'
tf.keras.utils.get_file(
    f'{model_name}.zip',
    f'https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip',
    cache_dir='.', cache_subdir='', extract=True
)

In [ ]:
tokenizers = tf.saved_model.load(model_name)

In [ ]:
[item for item in dir(tokenizers.en) if not item.startswith('_')]

In [ ]:
print('> This is a batch of strings:')
for en in en_examples.numpy():
  print(en.decode('utf-8'))

In [ ]:
encoded = tokenizers.en.tokenize(en_examples)

print('> This is a padded-batch of token IDs:')
for row in encoded.to_list():
  print(row)

In [ ]:
round_trip = tokenizers.en.detokenize(encoded)

print('> This is human-readable text:')
for line in round_trip.numpy():
  print(line.decode('utf-8'))

In [ ]:
print('> This is the text split into tokens:')
tokens = tokenizers.en.lookup(encoded)
tokens

A saída demonstra o aspecto de "subpalavra" da tokenização de subpalavras.

Por exemplo, a palavra `'searchability'` é decomposta em `'search'` e `'##ability'`, enquanto a palavra `'serendipity'` é decomposta em `'s'`, `'##ere'`, `'##nd'`, `'##ip'` e `'##ity'`.

Observe que o texto tokenizado inclui os tokens `'[START]'` e `'[END]'`.

In [ ]:
lengths = []

for pt_examples, en_examples in train_examples.batch(1024):
  pt_tokens = tokenizers.pt.tokenize(pt_examples)
  lengths.append(pt_tokens.row_lengths())

  en_tokens = tokenizers.en.tokenize(en_examples)
  lengths.append(en_tokens.row_lengths())
  print('.', end='', flush=True)

In [ ]:
all_lengths = np.concatenate(lengths)

plt.hist(all_lengths, np.linspace(0, 500, 101))
plt.ylim(plt.ylim())
max_length = max(all_lengths)
plt.plot([max_length, max_length], plt.ylim())
plt.title(f'Maximum tokens per example: {max_length}');

### Configurar um pipeline de dados com `tf.data`

A seguinte função recebe lotes de texto como entrada e os converte para um formato adequado para treinamento.

1. Ela tokeniza os textos em lotes irregulares.
2. Ela corta cada um para não ter mais do que `MAX_TOKENS`.
3. Ela separa os tokens de destino (inglês) em entradas e rótulos, deslocando-os uma etapa para que, em cada posição de entrada, o `rótulo` seja o ID do próximo token.
4. Ela converte os `RaggedTensor`s em `Tensor`s densos com preenchimento.
5. Ela retorna um par `(entradas, rótulos)`.

In [ ]:
MAX_TOKENS=64
def prepare_batch(pt, en):
    pt = tokenizers.pt.tokenize(pt)      # Output is ragged.
    pt = pt[:, :MAX_TOKENS]    # Trim to MAX_TOKENS.
    pt = pt.to_tensor()  # Convert to 0-padded dense Tensor

    en = tokenizers.en.tokenize(en)
    en = en[:, :(MAX_TOKENS+1)]
    en_inputs = en[:, :-1].to_tensor()  # Drop the [END] tokens
    en_labels = en[:, 1:].to_tensor()   # Drop the [START] tokens

    return (pt, en_inputs), en_labels

A função abaixo converte um conjunto de dados de exemplos de texto em dados em lotes para treinamento.

1. Ela tokeniza o texto e filtra as sequências que são muito longas.  
   (O uso de `batch`/`unbatch` é incluído porque o tokenizador é muito mais eficiente em grandes lotes).
2. O método `cache` garante que esse trabalho seja executado apenas uma vez.
3. Em seguida, `shuffle` e `dense_to_ragged_batch` randomizam a ordem e montam lotes de exemplos.
4. Por fim, `prefetch` executa o conjunto de dados em paralelo com o modelo para garantir que os dados estejam disponíveis quando necessário. Veja [Melhores desempenhos com o `tf.data`](https://www.tensorflow.org/guide/data_performance.ipynb) para mais detalhes.

In [ ]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [ ]:
def make_batches(ds):
  return (
      ds
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(prepare_batch, tf.data.AUTOTUNE)
      .prefetch(buffer_size=tf.data.AUTOTUNE))

 </section>

## Dataset de Teste

In [ ]:
# Create training and validation set batches.
train_batches = make_batches(train_examples)
val_batches = make_batches(val_examples)

In [ ]:
for (pt, en), en_labels in train_batches.take(1):
  break

print(pt.shape)
print(en.shape)
print(en_labels.shape)

The `en` and `en_labels` are the same, just shifted by 1:

In [ ]:
print(en[0][:10])
print(en_labels[0][:10])

## Definir os componentes

In [ ]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1)

  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
#@title
pos_encoding = positional_encoding(length=2048, depth=512)

# Check the shape.
print(pos_encoding.shape)

# Plot the dimensions.
plt.pcolormesh(pos_encoding.numpy().T, cmap='RdBu')
plt.ylabel('Depth')
plt.xlabel('Position')
plt.colorbar()
plt.show()

Por definição, esses vetores se alinham bem com vetores próximos ao longo do eixo de posição. Abaixo, os vetores de codificação de posição são normalizados, e o vetor da posição `1000` é comparado, por meio do produto escalar, a todos os outros:

In [ ]:
#@title
pos_encoding/=tf.norm(pos_encoding, axis=1, keepdims=True)
p = pos_encoding[1000]
dots = tf.einsum('pd,d -> p', pos_encoding, p)
plt.subplot(2,1,1)
plt.plot(dots)
plt.ylim([0,1])
plt.plot([950, 950, float('nan'), 1050, 1050],
         [0,1,float('nan'),0,1], color='k', label='Zoom')
plt.legend()
plt.subplot(2,1,2)
plt.plot(dots)
plt.xlim([950, 1050])
plt.ylim([0,1])


In [ ]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x


In [ ]:
embed_pt = PositionalEmbedding(vocab_size=tokenizers.pt.get_vocab_size().numpy(), d_model=248)
embed_en = PositionalEmbedding(vocab_size=tokenizers.en.get_vocab_size().numpy(), d_model=248)

pt_emb = embed_pt(pt)
en_emb = embed_en(en)

In [ ]:
en_emb._keras_mask

### Adicionar e normalizar

<table>
<tr>
  <th colspan=2>Adicionar e normalizar</th>
</tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/Add+Norm.png"/>
  </td>
</tr>
</table>

### A camada de atenção básica

Camadas de atenção são usadas em todo o modelo. Todas são idênticas, exceto pela configuração da atenção. Cada uma contém uma `layers.MultiHeadAttention`, uma `layers.LayerNormalization` e uma `layers.Add`.

<table>
<tr>
  <th colspan=2>A camada de atenção básica</th>
</tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/BaseAttention.png"/>
  </td>
</tr>
</table>

In [ ]:
class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

In [ ]:
class CrossAttention(BaseAttention):
  def call(self, x, context):
    attn_output, attn_scores = self.mha(
        query=x,
        key=context,
        value=context,
        return_attention_scores=True)

    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [ ]:
sample_ca = CrossAttention(num_heads=2, key_dim=248)

print(pt_emb.shape)
print(en_emb.shape)
print(sample_ca(en_emb, pt_emb).shape)

In [ ]:
class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [ ]:
sample_gsa = GlobalSelfAttention(num_heads=2, key_dim=248)

print(pt_emb.shape)
print(sample_gsa(pt_emb).shape)

Sticking with the same style as before you could draw it like this:

In [ ]:
class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [ ]:
sample_csa = CausalSelfAttention(num_heads=2, key_dim=248)

print(en_emb.shape)
print(sample_csa(en_emb).shape)

In [ ]:
out1 = sample_csa(embed_en(en[:, :3]))
out2 = sample_csa(embed_en(en))[:, :3]

tf.reduce_max(abs(out1 - out2)).numpy()

### A rede neural de feedforward

<table>
<tr>
  <th colspan=1>A rede neural de feedforward</th>
</tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/FeedForward.png"/>
  </td>
</tr>
</table>

A rede consiste em duas camadas lineares (`tf.keras.layers.Dense`) com uma ativação ReLU entre elas e uma camada de dropout. Assim como nas camadas de atenção, o código aqui também inclui a conexão residual e a normalização:

In [ ]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x)
    return x


In [ ]:
sample_ffn = FeedForward(248, 248)

print(en_emb.shape)
print(sample_ffn(en_emb).shape)

### O codificador

<table>
<tr>
  <th colspan=1>O codificador</th>
</tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/Encoder.png"/>
  </td>
</tr>
</table>

O codificador consiste em:

- Uma camada `PositionalEmbedding` na entrada.
- Um conjunto de camadas `EncoderLayer`.

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x

And a quick test, the output will have the same shape as the input:

In [ ]:
sample_encoder_layer = EncoderLayer(d_model=248, num_heads=4, dff=248)

print(pt_emb.shape)
print(sample_encoder_layer(pt_emb).shape)

### O codificador

<table>
<tr>
  <th colspan=1>O codificador</th>
</tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/Encoder.png"/>
  </td>
</tr>
</table>

O codificador consiste em:

- Uma camada `PositionalEmbedding` na entrada.
- Um conjunto de camadas `EncoderLayer`.

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads,
               dff, vocab_size, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(
        vocab_size=vocab_size, d_model=d_model)

    self.enc_layers = [
        EncoderLayer(d_model=d_model,
                     num_heads=num_heads,
                     dff=dff,
                     dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    # `x` is token-IDs shape: (batch, seq_len)
    x = self.pos_embedding(x)  # Shape `(batch_size, seq_len, d_model)`.

    # Add dropout.
    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x)

    return x  # Shape `(batch_size, seq_len, d_model)`.

In [ ]:
# Instantiate the encoder.
sample_encoder = Encoder(num_layers=2,
                         d_model=248,
                         num_heads=4,
                         dff=248,
                         vocab_size=8500)

sample_encoder_output = sample_encoder(pt, training=False)

# Print the shape.
print(pt.shape)
print(sample_encoder_output.shape)  # Shape `(batch_size, input_seq_len, d_model)`.

### A camada do decodificador

A pilha do decodificador é um pouco mais complexa, com cada `DecoderLayer` contendo uma `CausalSelfAttention`, uma `CrossAttention` e uma camada `FeedForward`:

<table>
<tr>
  <th colspan=1>A camada do decodificador</th>
</tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/DecoderLayer.png"/>
  </td>
</tr>
</table>

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.cross_attention = CrossAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context)

    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.cross_attention.last_attn_scores

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x

In [ ]:
sample_decoder_layer = DecoderLayer(d_model=248, num_heads=4, dff=248)

sample_decoder_layer_output = sample_decoder_layer(
    x=en_emb, context=pt_emb)

print(en_emb.shape)
print(pt_emb.shape)
print(sample_decoder_layer_output.shape)  # `(batch_size, seq_len, d_model)`

### O decodificador

Assim como o `Encoder`, o `Decoder` consiste em uma `PositionalEmbedding` e uma pilha de `DecoderLayer`s:

<table>
<tr>
  <th colspan=1>A camada de embedding e codificação posicional</th>
</tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/Decoder.png"/>
  </td>
</tr>
</table>

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size,
                                             d_model=d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

    self.last_attn_scores = None

  def call(self, x, context):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x, context)

    self.last_attn_scores = self.dec_layers[-1].last_attn_scores

    # The shape of x is (batch_size, target_seq_len, d_model).
    return x

In [ ]:
# Instantiate the decoder.
sample_decoder = Decoder(num_layers=2,
                         d_model=248,
                         num_heads=4,
                         dff=248,
                         vocab_size=8000)

output = sample_decoder(
    x=en,
    context=pt_emb)

# Print the shapes.
print(en.shape)
print(pt_emb.shape)
print(output.shape)

In [ ]:
sample_decoder.last_attn_scores.shape  # (batch, heads, target_seq, input_seq)

## O Transformer

Agora que você tem o `Encoder` e o `Decoder`, para completar o modelo `Transformer`, é necessário juntá-los e adicionar uma camada linear (`Dense`) final que converte o vetor resultante em cada localização em probabilidades de tokens de saída.

A saída do decodificador é a entrada para essa camada linear final.

<table>
<tr>
  <th colspan=1>O transformer</th>
</tr>
<tr>
  <td>
   <img src="https://www.tensorflow.org/images/tutorials/transformer/transformer.png"/>
  </td>
</tr>
</table>

In [ ]:
class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
    super().__init__()
    self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=input_vocab_size,
                           dropout_rate=dropout_rate)

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    context, x  = inputs

    context = self.encoder(context)  # (batch_size, context_len, d_model)

    x = self.decoder(x, context)  # (batch_size, target_len, d_model)

    # Final linear layer output.
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics.
      # b/250038731
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits

### Hiperparâmetros

In [ ]:
num_layers = 4
d_model = 124
dff = 248
num_heads = 4
dropout_rate = 0.1

### Instanciando Tranformer

In [ ]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=tokenizers.pt.get_vocab_size().numpy(),
    target_vocab_size=tokenizers.en.get_vocab_size().numpy(),
    dropout_rate=dropout_rate)

In [ ]:
output = transformer((pt, en))

print(en.shape)
print(pt.shape)
print(output.shape)

In [ ]:
attn_scores = transformer.decoder.dec_layers[-1].last_attn_scores
print(attn_scores.shape)  # (batch, heads, target_seq, input_seq)

## Treino


### Setar otimizador


In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [ ]:
plt.plot(learning_rate(tf.range(40000, dtype=tf.float32)))
plt.ylabel('Learning Rate')
plt.xlabel('Train Step')

### Função de perda e métricas


In [ ]:
def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

### Treinando

In [ ]:
transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
transformer.fit(train_batches,
                epochs=10,
                validation_data=val_batches,
                callbacks=[early_stopping])
